# Financial Data Processing Pipeline

## Overview
This notebook processes financial transactions from CSV to SQL Server, including data cleaning and transformation.

## Process Flow
1. Setup
    - Install required packages
2. Data Import & Transform
    - Load CSV data
    - Clean and standardize fields 
    - Convert data types
3. Database Operations  
    - SQL Server connection
    - Insert transformed data

## Data Structure
- Transaction details (dates, amounts)
- Account information 
- Categories and metadata


In [ ]:
import sys
!{sys.executable} -m pip install pandas pyodbc

In [ ]:
# Import Required Libraries
import pandas as pd
import pyodbc

In [ ]:
# Read the File
file_path = 'c:\\Users\\owner\\Downloads\\transactions.csv'  # Replace with your file path
df = pd.read_csv(file_path, dtype=str)

# Display the first few rows of the DataFrame to verify
# df.head()

In [ ]:
# Rename Columns
df.columns = ['date', 'original_date', 'account_type', 'account_name', 'account_number', 'institution_name', 'name', 'custom_name', 'amount', 'description', 'category', 'note', 'ignored_from', 'tax_deductible']

# Display the first few rows of the DataFrame to verify the column renaming
# df.head()

In [ ]:
# Rename Specific Values in Columns
df['account_name'] = df['account_name'].replace({
    'BEFORE': 'AFTER',
    'BEFORE': 'AFTER',
    'BEFORE': 'AFTER'
})

# Display the first few rows of the DataFrame to verify the value renaming
# df.head()

In [ ]:
# # Print column types to verify
# print("DataFrame column types:")
# print(df.dtypes)

# Convert date columns to datetime
df['date'] = pd.to_datetime(df['date'])
df['original_date'] = pd.to_datetime(df['original_date'])

# Convert amount to decimal
df['amount'] = pd.to_numeric(df['amount'])

# Replace NaN values with empty strings for text fields
text_columns = ['custom_name', 'note', 'ignored_from', 'tax_deductible']
df[text_columns] = df[text_columns].fillna('')

# Ensure string columns don't exceed VARCHAR(255)
varchar_columns = ['account_type', 'account_name', 'account_number', 'institution_name', 'name', 'custom_name', 'category', 'ignored_from', 'tax_deductible']
df[varchar_columns] = df[varchar_columns].apply(lambda x: x.str[:255])

# # Print updated column types
# print("\nUpdated DataFrame column types:")
# print(df.dtypes)

In [ ]:
# for driver in pyodbc.drivers():
#     print(driver)

In [ ]:
# Create SQL Server connection using pyodbc
conn = pyodbc.connect(
    'DRIVER={ODBC Driver 17 for SQL Server};'
    'SERVER=(localdb)\\finances;'
    'DATABASE=finances;'
    'Trusted_Connection=yes;'
)

try:
    # Create a cursor
    cursor = conn.cursor()
    
    # Delete all existing records
    cursor.execute("DELETE FROM rocketmoney_transactions")
    print("All existing records deleted.")
    
    # Insert data row by row
    for _, row in df.iterrows():
        cursor.execute("""
            INSERT INTO rocketmoney_transactions 
            VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
        """, tuple(row))
    
    # Commit the transaction
    conn.commit()
    print("Data successfully inserted into the database!")

except Exception as e:
    print(f"An error occurred: {str(e)}")
    conn.rollback()

finally:
    # Close the connection
    conn.close()